# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#importing all the dependencies 
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint 
from azureml.core import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AzureCapstone'

experiment=Experiment(ws, experiment_name)

In [3]:
#registering dataset on Azure Storage
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/SaadMuhammad/Azure_Capstone/main/default_credit_clients1.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
df = ds.to_pandas_dataframe()

In [7]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
#creating compute cluster 
compute_name = "SaadCompute"
try:
    amlcluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    amlcluster = ComputeTarget.create(ws, compute_name, compute_config)
amlcluster.wait_for_completion(show_output=True)

SaadCompute exists already

Running


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [27]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here, since i will be deploying a ONNX model, enable_onnx is set to true
automl_config = AutoMLConfig(
    task='classification',
    compute_target=amlcluster,
    training_data=ds,
    label_column_name='default payment next month',
    enable_onnx_compatible_models=True,
    n_cross_validations=4,
    **automl_settings)

In [28]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_e11d8e10-66de-47f1-ab0d-a1bb5f94d310',
 'target': 'Saadcompute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-29T15:15:43.835907Z',
 'endTimeUtc': '2021-01-29T15:50:10.677994Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'Saadcompute',
  'AMLSettingsJsonString': '{"path":null,"name":"AzureCapstone","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-136503","workspace_name":"quick-starts-ws-136503","region":"southcentralus","compute_target":"Saadcompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y_max":null,"num_classes":null,"f

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [30]:
# best run and model for automl
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# All metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: AzureCapstone,
Id: AutoML_e11d8e10-66de-47f1-ab0d-a1bb5f94d310_58,
Type: azureml.scriptrun,
Status: Completed)
precision_score_micro 0.8225666666666667
recall_score_weighted 0.8225666666666667
AUC_weighted 0.7877722126322521
log_loss 0.42496382301240343
precision_score_weighted 0.8067262255728733
recall_score_macro 0.6589292356218968
average_precision_score_weighted 0.8398767799247577
average_precision_score_macro 0.7426551665695387
weighted_accuracy 0.908591195206147
AUC_micro 0.8856852888888889
accuracy 0.8225666666666667
AUC_macro 0.787772219708421
balanced_accuracy 0.6589292356218968
average_precision_score_micro 0.8779486873416391
f1_score_micro 0.8225666666666667
precision_score_macro 0.7635345892688461
f1_score_weighted 0.8010161756924329
f1_score_macro 0.6848706132502604
norm_macro_recall 0.31785847124379357
matthews_correlation 0.4092127727707982
recall_score_micro 0.8225666666666667
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_e11d8e10-66de-47f1-a

In [31]:
# All the properties of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['0', '38', '43', '28', '14', '34', '51', '39', '44'],
 'weights': [0.13333333333333333,
             0.13333333333333333,
             0.13333333333333333,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.2,
             0.06666666666666667]}

0 - maxabsscaler
{'copy': True}

0 - lightgbmclassifier
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,


In [32]:
#TODO: Save the best model
Cap_Model = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneAzure_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(Cap_Model)

Model(workspace=Workspace.create(name='quick-starts-ws-136503', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-136503'), name=capstoneAzure_automl, id=capstoneAzure_automl:2, version=2, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.8225666666666667'})


In [33]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

capstoneAzure_automl version: 2
	 Training context : Auto ML
	 Accuracy : 0.8225666666666667


capstoneAzure_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.8223666666666667




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [34]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

In [35]:
# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [36]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 1)
service = Model.deploy(ws, "saadservice", [Cap_Model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://a5131e47-db0c-46b3-ae4a-3785096db396.southcentralus.azurecontainer.io/score
http://a5131e47-db0c-46b3-ae4a-3785096db396.southcentralus.azurecontainer.io/swagger.json


In [37]:

import json

#Import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
labels = test_df.pop('default payment next month')

sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(sample)

{"data": [{"ID": 29117, "LIMIT_BAL": 20000, "SEX": 2, "EDUCATION": 3, "MARRIAGE": 2, "AGE": 54, "PAY_0": 0, "PAY_2": 0, "PAY_3": 0, "PAY_4": 0, "PAY_5": 0, "PAY_6": 0, "BILL_AMT1": 13543, "BILL_AMT2": 14308, "BILL_AMT3": 16856, "BILL_AMT4": 18073, "BILL_AMT5": 18773, "BILL_AMT6": 19166, "PAY_AMT1": 1300, "PAY_AMT2": 2800, "PAY_AMT3": 1500, "PAY_AMT4": 1000, "PAY_AMT5": 695, "PAY_AMT6": 1000}, {"ID": 5504, "LIMIT_BAL": 80000, "SEX": 2, "EDUCATION": 1, "MARRIAGE": 2, "AGE": 31, "PAY_0": -1, "PAY_2": -1, "PAY_3": -1, "PAY_4": -1, "PAY_5": -1, "PAY_6": 2, "BILL_AMT1": 14401, "BILL_AMT2": 4374, "BILL_AMT3": 15020, "BILL_AMT4": 4233, "BILL_AMT5": 5934, "BILL_AMT6": 158, "PAY_AMT1": 4374, "PAY_AMT2": 15037, "PAY_AMT3": 4233, "PAY_AMT4": 6092, "PAY_AMT5": 0, "PAY_AMT6": 13391}, {"ID": 10572, "LIMIT_BAL": 100000, "SEX": 2, "EDUCATION": 3, "MARRIAGE": 2, "AGE": 41, "PAY_0": -1, "PAY_2": -1, "PAY_3": -1, "PAY_4": -1, "PAY_5": 0, "PAY_6": -1, "BILL_AMT1": 7432, "BILL_AMT2": 1633, "BILL_AMT3": 2043

TODO: In the cell below, send a request to the web service you deployed to test it.

In [38]:
import requests # for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, sample, headers=headers)

In [39]:
# Print results from the inference
print(response.text)

"{\"result\": [0, 0, 0, 0, 0]}"


In [40]:
# Print original labels
print(labels)

29116    0
5503     0
10571    1
3390     1
1299     1
Name: default payment next month, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [41]:
print(service.get_logs())

2021-01-29T15:57:27,092414387+00:00 - iot-server/run 
2021-01-29T15:57:27,093218691+00:00 - gunicorn/run 
2021-01-29T15:57:27,098029418+00:00 - rsyslog/run 
2021-01-29T15:57:27,101834538+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

## Converting model to ONNX Format

In [42]:
#Set the parameter return_onnx_model=True to retrieve the best ONNX model, instead of the Python model
best_run, onnx_model = remote_run.get_output(return_onnx_model=True)

In [44]:
#Save the best ONNX model
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_file_path = "./saad_onnx_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, onnx_file_path)

In [45]:
#deleting the service as per requirement
service.delete()